In [48]:
pip uninstall -y dynamic-f-models dfm || true

Found existing installation: dynamic-f-models 0.1.0
Uninstalling dynamic-f-models-0.1.0:
  Successfully uninstalled dynamic-f-models-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install --no-cache-dir "git+https://github.com/PSW1998/dfm.git@main"

  Cloning https://github.com/PSW1998/dfm.git (to revision main) to /private/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/pip-req-build-bwp5nzhr
  Running command git clone --filter=blob:none --quiet https://github.com/PSW1998/dfm.git /private/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/pip-req-build-bwp5nzhr
  Resolved https://github.com/PSW1998/dfm.git to commit f8465660f5eb0600bc0eaa906c5a8a18ac568d4f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dynamic-f-models: filename=dynamic_f_models-0.1.0-py3-none-any.whl size=2580 sha256=79ffe9764d78691c31fc69b201a0b226dfe7cfde15ab8edadf44fe20efd2af44
  Stored in directory: /private/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/pip-ephem-wheel-cache-l0ie380w/wheels/0f/9e/87/474f5875d646268eb92b15308e6229f38c3c698a7710eb3190
Successfully built dynamic-f-models
Note: you may need to restart the kernel to use updated packages.


In [50]:
import numpy as np
import pandas as pd
import yfinance as yf
from dfm.inversegamma import fit_igdf_hyperparams

raw = yf.download("SPY", start="2020-07-01", end="2025-07-01", interval="1d")
df = raw.xs('SPY', axis=1, level=-1).copy()
df = df[['Open','High','Low','Close','Volume']]  # ensure order

# your RS-style variance (using logs) + overnight jump
df_log = np.log(df)
df_log['PrevClose'] = df_log['Close'].shift(1)
df_log['Var'] = (
    (df_log['High'] - df_log['Open']) * (df_log['High'] - df_log['Close']) +
    (df_log['Open'] - df_log['Low']) * (df_log['High'] - df_log['Low']) +
    (df_log['Open'] - df_log['PrevClose'])**2
)
y = df_log['Var'].dropna().clip(lower=1e-12).to_numpy()

/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/ipykernel_7358/4256824646.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download("SPY", start="2020-07-01", end="2025-07-01", interval="1d")
[*********************100%***********************]  1 of 1 completed


In [51]:
# assuming your functions are available in the current namespace,
# otherwise: from dynagamma import fit_igdf_hyperparams
res = fit_igdf_hyperparams(
    y,
    alpha_grid=np.linspace(1.0, 2.5, 21),
    beta_grid=np.linspace(0.60, 0.98, 33),
    n0=49.0, s0=1/5000.0,
    warmup=630,                # optional: drop first 630 obs
    return_surface=True        # needed for plotting
)

print(f"alpha* = {res.alpha_star:.4f}")
print(f"beta*  = {res.beta_star:.4f}")
print(f"max loglik = {res.max_loglik:.2f}")


alpha* = 1.4500
beta*  = 0.7544
max loglik = 5043.19
